In [ ]:
pip install tensorflow

In [10]:
import cv2
import os

def extract_faces(video_path, output_folder, label):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(video_path)
    # frame_rate = cap.get()  # Frame rate
    count = 0

    while cap.isOpened():
        # frame_id = cap.get(1)  # Current frame number
        ret, frame = cap.read()

        if not ret:
            break

        # if frame_id % int(frame_rate) == 0:
            # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Extract and resize the face region
            face = frame[y:y + h, x:x + w]
            resized_face = cv2.resize(face, (224, 224))

            # Save the resized face
            output_path = os.path.join(output_folder, f"{label}_face_{count}.jpg")
            cv2.imwrite(output_path, resized_face)

            count += 1
            # Skip frames based on frame rate
            # frames_to_skip = int(frame_rate) - 1
            # for _ in range(frames_to_skip):
            #     cap.read()

    cap.release()
    

In [12]:

# Example usage:
real_video_path = "original"
fake_video_path = "fake"
og_folder = "og_images"
fake_folder = "fake_images"

# Extract faces from real videos
for video_file in os.listdir(real_video_path):
    video_path = os.path.join(real_video_path, video_file)
    extract_faces(video_path, og_folder, label="real")

# Extract faces from fake videos
for video_file in os.listdir(fake_video_path):
    video_path = os.path.join(fake_video_path, video_file)
    extract_faces(video_path, fake_folder, label="fake")


# Vinni

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array


c:\Users\nikhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10


In [4]:

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img_array = img_to_array(img)
        images.append(img_array)
        label = filename.split('_')[0]
        labels.append(label)
    return np.array(images), np.array(labels)


In [5]:

# Load and preprocess images
# real_images, real_labels = load_images_from_folder("path/to/preprocessed_faces/real")
# fake_images, fake_labels = load_images_from_folder("path/to/preprocessed_faces/fake")

images, labels=load_images_from_folder("outputimages")
# Combine real and fake data
# images = np.concatenate((real_images, fake_images), axis=0)
# labels = np.concatenate((real_labels, fake_labels), axis=0)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)


In [6]:

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)


In [7]:

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))





Epoch 1/10


1/1 [==============================] - 2s 2s/step - loss: 9.1206 - accuracy: 0.0000e+00 - val_loss: 2482.8218 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 173ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 4710.0161 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 179ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 6887.2910 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 200ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 9021.4766 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 183ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 11109.9531 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 167ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 13127.8984 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 167ms/step - loss: 0.0000e+00 - accur

ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter

# Nick

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Image dimensions
img_width, img_height = 64, 64  # Adjust based on your dataset

train_data_dir = 'images/'  # Path to the parent folder containing 'original' and 'fake' folders
batch_size = 32

# Data generator for training
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # use 'softmax' for more than two classes
])


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('deepfake_detection_model.h5')

Found 742 images belonging to 2 classes.
Found 185 images belonging to 2 classes.
Epoch 1/10
23/23 [==============================] - 4s 119ms/step - loss: 0.3652 - accuracy: 0.8521 - val_loss: 0.0122 - val_accuracy: 1.0000
Epoch 2/10
23/23 [==============================] - 2s 86ms/step - loss: 0.0363 - accuracy: 0.9845 - val_loss: 3.0825e-04 - val_accuracy: 1.0000
Epoch 3/10
23/23 [==============================] - 2s 87ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 4.0371e-05 - val_accuracy: 1.0000
Epoch 4/10
23/23 [==============================] - 2s 88ms/step - loss: 2.2781e-04 - accuracy: 1.0000 - val_loss: 1.9535e-05 - val_accuracy: 1.0000
Epoch 5/10
23/23 [==============================] - 2s 90ms/step - loss: 2.3154e-04 - accuracy: 1.0000 - val_loss: 1.8219e-05 - val_accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 2s 87ms/step - loss: 5.8794e-05 - accuracy: 1.0000 - val_loss: 7.9766e-06 - val_accuracy: 1.0000
Epoch 7/10
23/23 [=====================

In [16]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model

def extract_frames(video_path, size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, size)
        frame = img_to_array(frame)
        frame = np.expand_dims(frame, axis=0)
        frames.append(frame)
    cap.release()
    return np.vstack(frames)

def predict_video(video_path, model):
    frames = extract_frames(video_path)
    # Assuming that your model was trained with normalized images
    frames = frames.astype('float32') / 255.0
    predictions = model.predict(frames)
    avg_prediction = np.mean(predictions)
    return 'FAKE' if avg_prediction > 0.5 else 'REAL'

# Load your trained model
model = load_model('deepfake_detection_model.h5')

# Path to the new video
new_video_path = 'id9_0009.mp4'

# Predict
video_label = predict_video(new_video_path, model)
print(f"The video is predicted as: {video_label}")


10/10 [==============================] - 0s 28ms/step
The video is predicted as: REAL
